In [1]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from nltk.cluster import KMeansClusterer, cosine_distance
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from numpy.random import shuffle
import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import math

In [40]:
def topic(file_path):
    raw_df = pd.read_csv(file_path)

    #raw_df.user_product=raw_df.user_product.apply(lambda x :x.replace(',','')).astype(int)
    #raw_df.helpful=raw_df.helpful.apply(lambda x :x.replace(',','')).astype(int)
    #raw_df.funny=raw_df.funny.apply(lambda x :x.replace(',','')).astype(int)
    #raw_df.game_time=raw_df.game_time.apply(lambda x :x.replace(',','')).astype(float)
    raw_df['review_len']=raw_df.review.apply(lambda x: len(x))

    #filter_condition = (raw_df.game_time > 60) & (raw_df.helpful > 0) & (raw_df.review_len > 10)
    #filtered_df = raw_df[filter_condition]
    filtered_df = raw_df
    filtered_df.shape

    # get bigrams
    text = []
    tagged_tokens = []
    for doc in filtered_df['review']:
        tokens=nltk.word_tokenize(doc)
        temp_tokens= nltk.pos_tag(tokens)
        for x in temp_tokens:
            tagged_tokens.append(x)

    bigrams=list(nltk.bigrams(tagged_tokens))
    for (x,y) in bigrams:
        if x[1].startswith('J') and y[1].startswith('N'):
            text.append(x[0] + " " + y[0])
            
            
    #text = list(filtered_df['review'])
    print(text[0])
    print(tagged_tokens[0])
    
    stop_words = stopwords.words('english')
    
    other_words = ["game",'divinity','original',"sin",'play','use','make','really','thing','get','go','would','early','access']
    stop_words += other_words
    
    # create tf-idf
    tf_vectorizer = CountVectorizer(max_df = 0.90, min_df = 5, stop_words = stop_words)
    tf = tf_vectorizer.fit_transform(text)
    tf_feature_names = tf_vectorizer.get_feature_names()
    print(tf_feature_names[0:10])
    print(tf.shape)

    from sklearn.model_selection import train_test_split
    tf_train, tf_test = train_test_split(tf, test_size=0.3, random_state=0)
    
    
    num_topics = 10

    # Run LDA. For details, check
    # http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html#sklearn.decomposition.LatentDirichletAllocation.perplexity

    # max_iter control the number of iterations 
    # evaluate_every determines how often the perplexity is calculated
    # n_jobs is the number of parallel threads
    lda = LatentDirichletAllocation(n_components=num_topics, \
                                max_iter=30,verbose=1,
                                evaluate_every=1, n_jobs=1,
                                random_state=0).fit(tf_train)
    
    
    # lda for test set
    topic_assign_test = lda.transform(tf_test)

    # find the topic with top one highest possibility
    top_one_topic = [doc.argsort()[-1] for doc in topic_assign_test]

    top_one_topic
    
    
    num_top_words=20

    # lda.components_ returns a KxN matrix
    # for word distribution in each topic.
    # Each row consists of 
    # probability (counts) of each word in the feature space

    for topic_idx, topic in enumerate(lda.components_):
        #print ("Topic %d:" % (topic_idx))
        # print out top 20 words per topic 
        words=[(tf_feature_names[i],topic[i]) \
               for i in topic.argsort()[::-1][0:num_top_words]]
        #print(words)
        #print("\n")
        
    
    

    num_top_words=50
    f, axarr = plt.subplots(5, 2, figsize=(20, 20));

    for topic_idx, topic in enumerate(lda.components_):
        # create a dataframe with two columns (word, weight) for each topic

        # create a word:count dictionary
        f={tf_feature_names[i]:topic[i] for i in topic.argsort()[::-1][0:num_top_words]}

        # generate wordcloud in subplots
        wordcloud = WordCloud(width=480, height=450, margin=0, background_color="black");
        _ = wordcloud.generate_from_frequencies(frequencies=f);

        _ = axarr[math.floor(topic_idx/2), topic_idx%2].imshow(wordcloud, interpolation="bilinear");
        _ = axarr[math.floor(topic_idx/2), topic_idx%2].set_title("Topic: "+str(topic_idx));
        _ = axarr[math.floor(topic_idx/2), topic_idx%2].axis('off')

    plt.tight_layout()
    plt.show()

if __name__ = '__main__':
    file_path1 = 'Patch_1.csv'
    file_path2 = 'Patch_2.csv'
    file_path3 = 'Patch_3.csv'
    file_path4 = 'Patch_4.csv'
    topic(file_path1)
    topic(file_path2)
    topic(file_path3)
    topic(file_path4)